In [1]:
# -*- coding: utf-8 -*-
import warnings
warnings.filterwarnings('ignore')

import csv, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.externals import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

pd.set_option('display.max_columns', 500)
curr_dir = '/home/souche/qiongjiu/hgc/'


In [15]:
data = pd.read_csv(curr_dir+"2020-11-02版带count处理完成的零售数据m.csv", header = 0, low_memory=False)
data.info(null_counts = True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1701843 entries, 0 to 1701842
Data columns (total 61 columns):
car_id                      1701843 non-null int64
brand_code                  1701606 non-null object
brand_name                  1701606 non-null object
series_code                 1701606 non-null object
series_name                 1701606 non-null object
model_code                  1701843 non-null object
model_name                  1701606 non-null object
mileage_std                 1701843 non-null float64
color_name                  1701766 non-null object
first_license_plate_date    1701843 non-null object
province_name               1701843 non-null object
city_name                   1701843 non-null object
price                       1701843 non-null float64
pay_time                    1701843 non-null object
license_month               1701843 non-null int64
residual                    1701843 non-null float64
days                        1701843 non-null int64
lic

In [16]:
car = pd.read_csv(curr_dir+"2020-11-02版车型参数及独热编码.csv", header = 0, low_memory=False)
car=car.drop(['model_year','new_car_price', 'driving_mode','gear_box_type','country_id',
 'import_type','intake_type', 'fuel_form','quality_mile', 'quality_year',
 'year_1','year_2','year_3','year_4','year_5','year_6','year_7','year_8',
 'year_9', 'year_10','year_11','year_12','year_13','year_14','year_15',
 'year_16', 'car_body','rate','rate_count'],axis=1)
print(car.shape)


(65250, 151)


In [17]:
data = pd.merge(data, car, how="inner", on="model_code")
vec = pd.read_csv(curr_dir+"2020-11-02版-TF-IDF权重的词向量.csv", dtype={'model_code': str}, header = 0)
vec = vec.drop(['model_name'], axis=1)
print(vec.shape)

data = pd.merge(data, vec, how="inner", on="model_code")
print(data.shape)


(65250, 101)
(1701843, 311)


In [18]:
data_B = data[(data['level']=="B") | 
              (data['level']=="C") | 
              (data['level']=="D") | 
              (data['level']=="E")]
print("筛选出B级以上的车数据量为:%d" % data_B.shape[0])
print("筛选出B级以上的车的model_code为:%d" % len(data_B['model_code'].unique()))
print("包含的车型级别:", data_B['level'].unique())


筛选出B级以上的车数据量为:830370
筛选出B级以上的车的model_code为:10985
包含的车型级别: ['B' 'D' 'C' 'E']


In [19]:
data_X = data_B.drop(['car_id', 'brand_code', 'brand_name', 'series_code', 
                          'series_name', 'model_code', 'model_name',  'color_name', 
                          'first_license_plate_date', 'province_name', 'city_name', 
                          'price', 'pay_time', 'residual', 'mileage_std', 'license_month', 
                          'new_car_price', 'count', 'series_level', 'level','level0','level1',
                          'level2','level3','level4','level5','level6','level7','level8'], axis=1)
data_y = data_B['residual']

print(data_X.shape, len(data_y))


(830370, 282) 830370


In [20]:
import lightgbm as lgb

gbm = lgb.LGBMRegressor(boosting_type='gbdt',
                        objective='regression',
                        learning_rate=0.05,
                        n_estimators=3000,
                        max_depth=8,
                        num_leaves=150,
                        subsample=1.0,
                        colsample_bytree=0.3,
                        min_child_samples=80,
                        n_jobs=8,
                        verbose=1)

gbm.fit(data_X,
        data_y,
        verbose=True)


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.3,
       learning_rate=0.05, max_depth=8, min_child_samples=80,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=3000,
       n_jobs=8, num_leaves=150, objective='regression', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0, verbose=1)

In [26]:
data_B['ypred'] = gbm.predict(data_X)
data_B['err'] = abs(data_B['ypred'] - data_B['residual'])/data_B['residual']
data_B[data_B['err']>=0.1]



,car_id,brand_code,brand_name,series_code,series_name,model_code,model_name,mileage_std,color_name,first_license_plate_date,province_name,city_name,price,pay_time,license_month,residual,days,license_year,model_year,new_car_price,rate,rate_count,per_mile,year_err,province_name0,province_name1,province_name2,province_name3,province_name4,province_name5,province_name6,province_name7,province_name8,province_name9,province_name10,province_name11,province_name12,province_name13,province_name14,province_name15,province_name16,province_name17,province_name18,province_name19,province_name20,province_name21,province_name22,province_name23,province_name24,province_name25,province_name26,province_name27,province_name28,province_name29,province_name30,keep_value,quality,mile_log,license_month_log,new_car_price_log,count,wheel_base,length,height,width,max_torque,max_power,engine_volume_l,cylinder_number,series_level,level,seat_number_top,driving_mode0,driving_mode1,driving_mode2,driving_mode3,driving_mode4,driving_mode5,driving_mode6,driving_mode7,driving_mode8,driving_mode9,gear_box_type0,gear_box_type1,gear_box_type2,gear_box_type3,gear_box_type4,gear_box_type5,gear_box_type6,gear_box_type7,gear_box_type8,gear_box_type9,gear_box_type10,country_id0,country_id1,country_id2,country_id3,country_id4,country_id5,country_id6,country_id7,country_id8,country_id9,country_id10,country_id11,country_id12,country_id13,country_id14,import_type0,import_type1,intake_type0,intake_type1,intake_type2,intake_type3,intake_type4,intake_type5,intake_type6,intake_type7,fuel_form0,fuel_form1,fuel_form2,fuel_form3,fuel_form4,fuel_form5,fuel_form6,fuel_form7,fuel_form8,level0,level1,level2,level3,level4,level5,level6,level7,level8,car_body0,car_body1,car_body2,car_body3,car_body4,car_body5,car_body6,car_body7,car_body8,car_body9,car_body10,car_body11,series_level0,series_level1,series_level2,series_level3,series_level4,series_level5,series_level6,series_level7,series_level8,series_level9,series_level10,series_level11,series_level12,series_level13,series_level14,series_level15,series_level16,series_level17,series_level18,series_level19,series_level20,series_level21,series_level22,series_level23,series_level24,series_level25,series_level26,series_level27,series_level28,series_level29,series_level30,series_level31,series_level32,series_level33,series_level34,series_level35,series_level36,series_level37,series_level38,series_level39,series_level40,series_level41,series_level42,series_level43,series_level44,series_level45,series_level46,series_level47,series_level48,series_level49,series_level50,series_level51,series_level52,series_level53,series_level54,series_level55,series_level56,series_level57,series_level58,series_level59,series_level60,series_level61,series_level62,wordvec0,wordvec1,wordvec2,wordvec3,wordvec4,wordvec5,wordvec6,wordvec7,wordvec8,wordvec9,wordvec10,wordvec11,wordvec12,wordvec13,wordvec14,wordvec15,wordvec16,wordvec17,wordvec18,wordvec19,wordvec20,wordvec21,wordvec22,wordvec23,wordvec24,wordvec25,wordvec26,wordvec27,wordvec28,wordvec29,wordvec30,wordvec31,wordvec32,wordvec33,wordvec34,wordvec35,wordvec36,wordvec37,wordvec38,wordvec39,wordvec40,wordvec41,wordvec42,wordvec43,wordvec44,wordvec45,wordvec46,wordvec47,wordvec48,wordvec49,wordvec50,wordvec51,wordvec52,wordvec53,wordvec54,wordvec55,wordvec56,wordvec57,wordvec58,wordvec59,wordvec60,wordvec61,wordvec62,wordvec63,wordvec64,wordvec65,wordvec66,wordvec67,wordvec68,wordvec69,wordvec70,wordvec71,wordvec72,wordvec73,wordvec74,wordvec75,wordvec76,wordvec77,wordvec78,wordvec79,wordvec80,wordvec81,wordvec82,wordvec83,wordvec84,wordvec85,wordvec86,wordvec87,wordvec88,wordvec89,wordvec90,wordvec91,wordvec92,wordvec93,wordvec94,wordvec95,wordvec96,wordvec97,wordvec98,wordvec99,ypred,err
0,13643647,brand-49,丰田,series-1072,汉兰达,054186468,2013款 汉兰达 2.7L 两驱7座探索版,12.00,3,2012-09-01,四川,成都,11.25,2020-10-26,98,0.342153,66,2012,2013,32.88,4.2,12.0,1.47,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [38]:
def data3m_pinggu(data_df,col):
    total = data_df.shape[0]
    num_3 = data_df.loc[(data_df[col]>=-0.03 ) & (data_df[col]<=0.03 )].shape[0]
    print("P<3%: ",round(num_3/total,4))
    num_5 = data_df.loc[(data_df[col]>=-0.05 ) & (data_df[col]<=0.05 )].shape[0]
    print("P<5%: ",round(num_5/total,4))
    num_8 = data_df.loc[(data_df[col]>=-0.08 ) & (data_df[col]<=0.08 )].shape[0]
    print("P<8%: ",round(num_8/total,4))
    num_10 = data_df.loc[(data_df[col]>=-0.1 ) & (data_df[col]<=0.1 )].shape[0]
    print("P<10%: ",round(num_10/total,4))
    num_20 = data_df.loc[(data_df[col]>=-0.2 ) & (data_df[col]<=0.2 )].shape[0]
    print("P<20%: ",round(num_20/total,4))

data_B['err'] = abs(data_B['ypred']-data_B['residual'])/data_B['residual']
data3m_pinggu(data_B.loc[(data_B['pay_time'] > '2020-10-01')],'err')


P<3%:  0.4729
P<5%:  0.6681
P<8%:  0.8249
P<10%:  0.8806
P<20%:  0.9711


In [40]:
data_B = data_B[(data_B['err']<0.1) & (data_B['err']>-0.1)]
#打乱数据
data_B = shuffle(data_B)

data_X = data_B.drop(['car_id', 'brand_code', 'brand_name', 'series_code', 
                          'series_name', 'model_code', 'model_name',  'color_name', 
                          'first_license_plate_date', 'province_name', 'city_name', 
                          'price', 'pay_time', 'residual', 'mileage_std', 'license_month', 
                          'new_car_price', 'count', 'series_level', 'level','level0','level1',
                          'level2','level3','level4','level5','level6','level7','level8','ypred', 'err'], axis=1)
data_y = data_B['residual']

print(data_X.shape, len(data_y))

(706450, 282) 706450


In [42]:
for i in range(15):
    data_X = data_X.rename(columns={"country_id"+str(i):"Country_Id"+str(i)})

In [45]:
from sklearn2pmml.pipeline import PMMLPipeline
from sklearn2pmml import sklearn2pmml
import lightgbm as lgb

pipeline = PMMLPipeline([("regressor", lgb.LGBMRegressor(boosting_type = 'gbdt', 
                                                         objective = 'regression', 
                                                         learning_rate = 0.05, 
                                                         n_estimators = 3000, 
                                                         max_depth = 8, 
                                                         num_leaves = 150, 
                                                         subsample = 1.0, 
                                                         colsample_bytree = 0.3, 
                                                         min_child_samples = 80, 
                                                         n_jobs = 8, 
                                                         verbose = 1))])
pipeline.fit(data_X, data_y)

ph =  curr_dir+"enterprise2_BBB_level_dfc_retail.pmml"
sklearn2pmml(pipeline, ph, with_repr = True)


In [46]:
joblib.dump(pipeline,curr_dir+'B_level_dfc_retail_model_2020-11-02_')


['/home/souche/qiongjiu/hgc/B_level_dfc_retail_model_2020-11-02_']

In [47]:
# 上传到 oss上 
ph = curr_dir+"enterprise2_BBB_level_dfc_retail.pmml"
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
oss.put_file("algorithm/qiongjiu/valuation/2020-11-02_/enterprise2_BBB_level_dfc_retail.pmml", ph)


DAYU_HOME : /home/souche/projects/datacenter-etl-v2
[2020-11-11 16:58:47,785] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/Grammar.txt
[2020-11-11 16:58:47,812] {driver:120} INFO - Generating grammar tables from /usr/lib/python3.5/lib2to3/PatternGrammar.txt
[2020-11-11 16:58:49,159] {oss_hook:28} INFO - Done. Loaded the key algorithm/qiongjiu/valuation/2020-11-02_/enterprise2_BBB_level_dfc_retail.pmml .


In [48]:
# 上传到 oss上 
ph = curr_dir+"B_level_dfc_retail_model_2020-11-02_"
from dayu.hooks.oss_hook import OSSHook
oss = OSSHook("oss_algorithm")
oss.put_file("algorithm/qiongjiu/valuation/2020-11-02_/B_level_dfc_retail_model_2020-11-02", ph)

[2020-11-11 16:58:50,131] {oss_hook:28} INFO - Done. Loaded the key algorithm/qiongjiu/valuation/2020-11-02_/B_level_dfc_retail_model_2020-11-02 .


In [52]:
data_B['ypred'] = pipeline.predict(data_X)
data_B['err'] = abs(data_B['ypred'] - data_B['residual'])
data_B[data_B['err']>=0.1]
def data3m_pinggu(data_df,col):
    total = data_df.shape[0]
    num_3 = data_df.loc[(data_df[col]>=-0.03 ) & (data_df[col]<=0.03 )].shape[0]
    print("P<3%: ",round(num_3/total,4))
    num_5 = data_df.loc[(data_df[col]>=-0.05 ) & (data_df[col]<=0.05 )].shape[0]
    print("P<5%: ",round(num_5/total,4))
    num_8 = data_df.loc[(data_df[col]>=-0.08 ) & (data_df[col]<=0.08 )].shape[0]
    print("P<8%: ",round(num_8/total,4))
    num_10 = data_df.loc[(data_df[col]>=-0.1 ) & (data_df[col]<=0.1 )].shape[0]
    print("P<10%: ",round(num_10/total,4))
    num_20 = data_df.loc[(data_df[col]>=-0.2 ) & (data_df[col]<=0.2 )].shape[0]
    print("P<20%: ",round(num_20/total,4))

data_B['err'] = abs(data_B['ypred']-data_B['residual'])/data_B['residual']
data3m_pinggu(data_B.loc[(data_B['pay_time'] > '2020-09-01')],'err')


P<3%:  0.5639
P<5%:  0.78
P<8%:  0.9414
P<10%:  0.9837
P<20%:  0.9998
